In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from multicamera_acquisition.paths import DATA_DIR
from multicamera_acquisition.acquisition import acquire_video

/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/interfaces/arduino.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [4]:
save_location = DATA_DIR / 'tests'

In [5]:
camera_list = [
    {'name': 'top', 'serial': 22181547, 'brand':'flir', 'gain': 12, "display":True},
    {'name': 'side1', 'serial': 22181612, 'brand':'flir', 'gain': 12, "display":False},
]

In [ ]:
acquire_video(
    save_location,
    camera_list,
    framerate = 30,
    exposure_time = 2000,
    recording_duration_s = 10,
    serial_timeout_duration_s=0.1,
    overwrite=False,
    append_datetime=True,
)

INFO:root:Checking cameras...
INFO:root:Initializing cameras...
INFO:root:Camera top...


Process AcquisitionLoop-3:
Traceback (most recent call last):
  File "/home/dattalab/anaconda3/envs/flir_acq/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py", line 67, in run
    cam = get_camera(brand=self.brand, **self.camera_params)
  File "/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/interfaces/__init__.py", line 35, in get_camera
    cam.init()
  File "/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/interfaces/camera_flir.py", line 69, in init
    self.cam.Init()
  File "/home/dattalab/anaconda3/envs/flir_acq/lib/python3.10/site-packages/PySpin.py", line 33433, in Init
    return _PySpin._SWIG_CamPtr_Init(self)
_PySpin.SpinnakerException: Spinnaker: NULL pointer dereferenced. [-1015]
